# Lecture 2 : Exact Diagonalisation for a finite size scaling

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PhilipVinc/ComputationalQuantumPhysics/blob/main/Notebooks/1-IntroManyBody/2-quspin.ipynb)

## Problem Statement:

We will be studying the Transverse Field Ising Model in 1 Dimension

$$
H(J,h) =  J \sum_{\langle i,j\rangle} \sigma_i^z\sigma_j^z -h \sum_i \sigma^x_i 
$$

where J is the ferromagnetic nearest-neighbor coupling, and h is the transverse field.
Remember that by $\sigma_i^x$ we mean

$$
\sigma_i^x = \mathbb{I} \otimes \mathbb{I} \otimes \dots \otimes \sigma^x \otimes \mathbb{I} \dots,
$$
where identities are everywhere except at the site $i$.

Notice that if $h=0$ this is equivalent to the classical ising model.

Without loss of generality we will from now on consider $J=1$ and only vary $h$.

We know that the classical ising Model ($h=0$) has a phase transition if we increase the temperature from a ferromagnetic or antiferromagnetic phase to a disordered phase. 

** How do we identify this phase transition classically? What is the order parameter? **

From now on, we will be looking at the ground state, as well as the first few excited states, defined as

$$
H(h)|\epsilon_i(h)\rangle = \epsilon_i(h) |\epsilon_i(h)\rangle
$$

Notice that the 'classical' term in the hamiltonian, which is $\sum_{\langle i,j\rangle} \sigma_i^z\sigma_j^z$, does not commute with the 'transverse field' term $\sum_i \sigma^x_i $. 

** Can you say something about the ground state of those two different terms, separately? **

### Part 2. Using an estabilished code

**After** having tried yourself to do what discussed above and having seen what are the maximum system sizes you can reach, you should try to use a custom python package, [quspin](https://quspin.github.io/QuSpin/), which is a good python package to do exact diagonalisation.

This package is considerably faster because (i) it does not store the matrix itself, instead it merely knows how to apply it to a ket, and (ii) uses Lanczos diagonalisation with this object.
We will discuss this approach in more detail in our second lecture, but in the meantime, you should try it out.

NOTE: For unfortunate reasons, quspin only works on python 3.9 -> 3.12, but not on 3.13 or 3.14. So make sure you have a correct python version.

The quspin code to write the TFIM Hamiltonian is something like the following
```python
from quspin.operators import hamiltonian  # Hamiltonians and operators
from quspin.basis import (
    spin_basis_1d,
    spinless_fermion_basis_1d,
)  # Hilbert space spin basis

#
##### define model parameters #####
L = 14  # system size
J = 1.0  # spin zz interaction
h = np.sqrt(2)  # z magnetic field strength

zblovk = [1,1]
basis = spin_basis_1d(L=L, zblock=zblock)

h_field = [[-h, i] for i in range(L)]
J_zz = [[-J, i, (i + 1) % L] for i in range(L)]  # PBC
# define spin static and dynamic lists
static_spin = [["zz", J_zz], ["x", h_field]]  # static part of H

# build spin Hamiltonians
H_spin = hamiltonian(static_spin, [], basis=basis, dtype=np.float64)

# calculate spin energy levels
E_spin = H_spin.eigvalsh()
```

Things of note:
 - You need to define the `basis_spin` object, which defines a fast way to generate configurations of the hilbert space. See for example `basis.int_to_state(4)`. Mathematically this defines the computational basis B(H) from the lecture notes.
 - The hamiltonian is defined symbolically as a list of terms (`static_spin`) which includes both the `zz` terms and the `x` terms, each accompanied by the sites on which it is defined.
 - The matrix `H_spin` is stored in a special sparse format of quspin, which we will not be discussing in detail, but it's even more performant than `scipy.sparse`.
 - To diagonalise this matrix, you need to use `H_spin.eigvalsh()` or `H_spin.eigsh`.

#### Things to do:
    
  1.  Benchmark the time taken by quspin to (respectively) construct the hamiltonian and to diagonalise it.
        - Time the two tasks for increasing system sizes $N$ 
        - Consider the plot you did in point (4) of Part 1, where you benchmarked the same tasks for your sparse-matrix based code. Include this new data in that plot. What do we learn?

  2. Now repeat the same study you did for point (3) of Part 1, but this time on each of the 3 plots where the x axis is the transverse field `h` and the y axis is an observable (energy, magnetization and magnetization squared) report multiple curves, each for a different value of `L` from 4 to whatever largest size you can achieve.
        - Using Quspin should allow you to do it much faster then before, and to reach larger system sizes.
        - It's a good idea to make legible plots to not rely on the default color sequence of matplotlib, which is an insult to aestethics. Instead, pick a color map that is easier to read for example because bright colors are smaller systems and darker colors are larger systems (or vice-versa). Examples are `magma` or `viridis` colormap, among many others. 



In [ ]:
### Using an estabilished code

import numpy as np
from quspin.operators import hamiltonian  # Hamiltonians and operators
from quspin.basis import (
    spin_basis_1d,
    spinless_fermion_basis_1d,
)  # Hilbert space spin basis

#
##### define model parameters #####
L = 4  # system size
J = 1.0  # spin zz interaction
h = np.sqrt(2)  # z magnetic field strength

basis = spin_basis_1d(L=L)

h_field = [[-h, i] for i in range(L)]
J_zz = [[-J, i, (i + 1) % L] for i in range(L)]  # PBC
# define spin static and dynamic lists
static_spin = [["zz", J_zz], ["x", h_field]]  # static part of H

# build spin Hamiltonians
H_spin = hamiltonian(static_spin, [], basis=basis, dtype=np.float64)

# calculate spin energy levels
E_spin = H_spin.eigvalsh()
E_spin, psi_0 = H_spin.eigsh(k=1, which="SA")


Hermiticity check passed!
Symmetry checks passed!
